In [8]:
import os
from PyPDF2 import PdfReader
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI

# -----------------------------
# Cargar API Key
# -----------------------------
path = find_dotenv()
load_dotenv(path)
API_KEY = os.getenv("API_KEY")

print("API Key cargada:", "Sí" if API_KEY else "No encontrada")

# Crear cliente OpenAI
client = OpenAI(api_key=API_KEY)
print("Cliente OpenAI inicializado ✅")


API Key cargada: Sí
Cliente OpenAI inicializado ✅


## Leer PDF (simulación)

In [14]:
pdf_path = "data-science.pdf" 
pdf_text = None

if os.path.exists(pdf_path):
    reader = PdfReader(pdf_path)
    text_parts = [page.extract_text() for page in reader.pages if page.extract_text()]
    pdf_text = "\n".join(text_parts)
    print(f"PDF cargado ✅ ({len(reader.pages)} páginas)")
    print("Primeros 2,000 caracteres del PDF:\n", pdf_text[:2000])
else:
    print("⚠️ No se encontró el archivo PDF, continua sin PDF")


PDF cargado ✅ (10 páginas)
Primeros 2,000 caracteres del PDF:
 Data 
SciencePrograma
Descripción de la Carrera
 En esta carrera desarrollarás las competencias necesarias para insertarte 
en el campo del Análisis y la Ciencia de datos. Aprenderás acerca del proceso de 
examinar, limpiar, transformar y modelar datos con el objetivo de descubrir 
patrones y conocimiento útil para las organizaciones o emprendimientos en los 
que te insertes. Además, conocerás el campo de la Ciencia de datos, el cual 
combina elementos de las matemáticas, estadísticas y tecnología, para extraer 
información útil de los datos con los cuales trabaja. 
 Ambas disciplinas son indispensables para la toma de decisiones 
informadas en una variedad de industrias, como las finanzas, la medicina, la 
ciencia y la tecnología, entre muchas otras.
Tecnologías que emplearás:
PostgreSQL DBeaver Tableau    
 Public
Google      
SheetsPython Jupyter   
Notebook
Pandas Anaconda Numpy   
PandasPyplot
 Seaborn
Statsmodels Scik

## Historial de mensajes

In [24]:
session_state = {}

if "messages" not in session_state:
    session_state["messages"] = [
        {"role": "system", "content": "Eres un asistente en español. Usa el contenido del PDF si está disponible para responder."}
    ]

print("Historial inicial:")
for msg in session_state["messages"]:
    print(msg)

print(session_state)

Historial inicial:
{'role': 'system', 'content': 'Eres un asistente en español. Usa el contenido del PDF si está disponible para responder.'}
{'messages': [{'role': 'system', 'content': 'Eres un asistente en español. Usa el contenido del PDF si está disponible para responder.'}]}


## Simular entrada de Usuario

In [25]:
user_prompt = "Hazme una lista de todos los programas que se utilizarán"
session_state["messages"].append({"role": "user", "content": user_prompt})

print("Historial después de entrada del usuario:")
for msg in session_state["messages"]:
    print(msg)

Historial después de entrada del usuario:
{'role': 'system', 'content': 'Eres un asistente en español. Usa el contenido del PDF si está disponible para responder.'}
{'role': 'user', 'content': 'Hazme una lista de todos los programas que se utilizarán'}


### Message for the model

In [27]:
messages_for_llm = session_state["messages"].copy()

if pdf_text:
    messages_for_llm.insert(
        1,
        {"role": "system", "content": f"Contenido del PDF:\n\n{pdf_text[:1500]} ..."}
    )

print("Mensajes que se enviarán al modelo:")
for msg in messages_for_llm:
    print(msg["role"], ":", msg["content"][:120], "...")


Mensajes que se enviarán al modelo:
system : Eres un asistente en español. Usa el contenido del PDF si está disponible para responder. ...
system : Contenido del PDF:

Data 
SciencePrograma
Descripción de la Carrera
 En esta carrera desarrollarás las competencias nece ...
user : Hazme una lista de todos los programas que se utilizarán ...


## Llamada Open Ai

In [28]:
try:
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for_llm,
        temperature=0.2
    )
    answer = resp.choices[0].message.content
except Exception as e:
    answer = f"⚠️ Error al llamar OpenAI: {e}"

# Guardar respuesta en historial
session_state["messages"].append({"role": "assistant", "content": answer})

print("Respuesta del asistente:\n", answer)

Respuesta del asistente:
 Aquí tienes la lista de programas y tecnologías que se utilizarán en el programa de Data Science:

1. PostgreSQL
2. DBeaver
3. Tableau
4. Google Sheets
5. Python
6. Jupyter Notebook
7. Pandas
8. Anaconda
9. Numpy
10. Pyplot
11. Seaborn
12. Statsmodels
13. Scikit-learn


In [29]:
print("Historial final:")
for msg in session_state["messages"]:
    print(msg)

Historial final:
{'role': 'system', 'content': 'Eres un asistente en español. Usa el contenido del PDF si está disponible para responder.'}
{'role': 'user', 'content': 'Hazme una lista de todos los programas que se utilizarán'}
{'role': 'assistant', 'content': 'Aquí tienes la lista de programas y tecnologías que se utilizarán en el programa de Data Science:\n\n1. PostgreSQL\n2. DBeaver\n3. Tableau\n4. Google Sheets\n5. Python\n6. Jupyter Notebook\n7. Pandas\n8. Anaconda\n9. Numpy\n10. Pyplot\n11. Seaborn\n12. Statsmodels\n13. Scikit-learn'}
